<h3 style = "font-family: Cambria;"> Data </h3>

In [1]:
import pandas, numpy, time

data = pandas.read_csv("Data.csv", index_col = "Ticker")

data = data.dropna()

print(data)

           Inventory  Working Capital  Total Assets  Current Assets  \
Ticker                                                                
A       8.626667e+08     1.985333e+09  1.028800e+10    3.664000e+09   
AA      1.927000e+09     1.936000e+09  1.488033e+10    4.932000e+09   
AAL     1.896000e+09    -4.457000e+09  6.439700e+10    1.456667e+10   
AAN     7.530493e+08     5.378673e+08  1.551062e+09    8.538473e+08   
AAON    1.371427e+08     1.653597e+08  6.376970e+08    2.624823e+08   
...              ...              ...           ...             ...   
ZIM     1.206457e+08     1.329239e+09  8.097114e+09    3.519343e+09   
ZTO     5.885600e+07     8.905596e+09  6.683356e+10    2.182200e+10   
ZTS     1.965333e+09     4.637667e+09  1.414467e+10    7.015667e+09   
ZWS     2.895667e+08     5.038000e+08  2.522933e+09    8.322000e+08   
ZYXI    1.095833e+07     5.373233e+07  1.073050e+08    7.175700e+07   

        Current Liabilities  Cash And Cash Equivalents    Total Debt  \
Tick

<h3 style = "font-family: Cambria;"> Normalization </h3>

In [2]:
from sklearn.preprocessing import MinMaxScaler

train = MinMaxScaler().fit_transform(data)

train = pandas.DataFrame(train, index = data.index)

print(train)

              0         1             2         3             4   \
Ticker                                                             
A       0.000102  0.489336  3.866041e-05  0.000067  2.423393e-05   
AA      0.000228  0.489335  5.594979e-05  0.000091  4.328614e-05   
AAL     0.000225  0.489124  2.423718e-04  0.000268  2.750896e-04   
AAN     0.000089  0.489289  5.767295e-06  0.000016  4.525782e-06   
AAON    0.000016  0.489276  2.328629e-06  0.000005  1.360512e-06   
...          ...       ...           ...       ...           ...   
ZIM     0.000014  0.489315  3.041209e-05  0.000065  3.163069e-05   
ZTO     0.000007  0.489564  2.515450e-04  0.000402  1.867621e-04   
ZTS     0.000233  0.489424  5.318012e-05  0.000129  3.434818e-05   
ZWS     0.000034  0.489287  9.426230e-06  0.000015  4.705409e-06   
ZYXI    0.000001  0.489273  3.317910e-07  0.000001  2.165405e-07   

                  5             6         7             8             9   \
Ticker                                 

<h3 style = "font-family: Cambria;"> Trace Maximization Algorithm </h3>

In [3]:
def vector_v(mat_x, mat_z):

    vec_z = numpy.transpose(mat_z)
    
    vec_v = numpy.array([])

    for i in vec_z:

        value = numpy.sqrt(i @ numpy.transpose(mat_x) @ mat_x @ i)

        value = value * ((i @ i) ** -1)
        
        vec_v = numpy.append(vec_v, value)

    return vec_v

In [4]:
def matrix_q(mat_x, mat_z):

    vec_z = numpy.transpose(mat_z)

    mat_q = numpy.array([])

    for i in vec_z:

        value = numpy.sqrt(i @ numpy.transpose(mat_x) @ mat_x @ i) ** -1
        
        value = (numpy.transpose(mat_x) @ mat_x @ i) * value
    
        mat_q = numpy.append(mat_q, value)
    
    return numpy.transpose(numpy.reshape(mat_q, (len(vec_z), -1)))

In [5]:
def matrix_d(mat_x, mat_q, vec_v):

    mat_v = numpy.zeros((len(vec_v), len(vec_v)))

    for i in range(len(vec_v)):

        mat_v[i][i] = vec_v[i]

    mat_a = 2 * mat_q @ mat_v

    mat_o = mat_v ** 2
    
    mat_o = numpy.ones((len(numpy.transpose(mat_x)), len(mat_v))) @ mat_o

    return mat_a - mat_o

In [6]:
def update_z(mat_x, index, k):

    diffs = list(set(numpy.arange(k)) - set(numpy.unique(index)))

    for i in range(len(diffs)):
    
        index[i] = diffs[i]
    
    mat_z = numpy.zeros((len(numpy.transpose(mat_x)), k))

    for i in range(len(index)):

        mat_z[i][index[i]] = 1

    return mat_z

In [7]:
def trace_maximization(mat_x, k):

    index = numpy.random.choice(k, len(numpy.transpose(mat_x)))

    mat_z = numpy.zeros((len(numpy.transpose(mat_x)), k))

    for i in range(len(index)):

        mat_z[i][index[i]] = 1

    vec_v = vector_v(mat_x, mat_z); mat_q = matrix_q(mat_x, mat_z)
    
    m_a_b = matrix_d(mat_x, mat_q, vec_v)

    for i in range(300):
        
        upd_z = update_z(mat_x, numpy.argmax(m_a_b, axis = 1), k)

        for j in range(100):

            mat_q = matrix_q(mat_x, upd_z); m_a_b = matrix_d(mat_x, mat_q, vec_v)

            new_z = update_z(mat_x, numpy.argmax(m_a_b, axis = 1), k)

            if numpy.linalg.norm(new_z - upd_z) < 0.00001:

                upd_z = new_z

                break
            
            upd_z = new_z
        
        if numpy.linalg.norm(upd_z - mat_z) < 0.00001:

            return upd_z
        
        mat_z = upd_z

        vec_v = vector_v(mat_x, mat_z)

    return mat_z

<h3 style = "font-family: Cambria;"> Result </h3>

In [8]:
length, result = time.time(), {}

for i in range(500):
    
    temps = trace_maximization(numpy.array(numpy.transpose(train)), 7)

    index = numpy.argmax(temps, axis = 1)

    dataframes = pandas.concat([train, pandas.DataFrame(index, columns = ["Clusters"], index = train.index)], axis = 1)
    
    dataframes = dataframes.groupby("Clusters").var(ddof = 0)

    count = numpy.unique(index, return_counts = True)[1]

    result[i] = [numpy.sum(count @ dataframes), index]

length = time.time() - length

In [9]:
index = numpy.argmin([result[i][0] for i in result])

colum = ["Clusters"]

frame = pandas.DataFrame(result[index][1], columns = colum, index = train.index)

print(frame)

        Clusters
Ticker          
A              5
AA             0
AAL            2
AAN            0
AAON           0
...          ...
ZIM            0
ZTO            2
ZTS            5
ZWS            5
ZYXI           2

[1152 rows x 1 columns]
